In [1]:
import scipy as sp
import numpy as np
import time

try:
    from localgraphclustering import *
except:
    # when the package is not installed, import the local version instead. 
    # the notebook must be placed in the original "notebooks/" folder
    sys.path.append("../")
    from localgraphclustering import * 

import time

## Load data

In [2]:
g = GraphLocal('./datasets/BlogCatalog-dataset/data/edges2.txt','edgelist',' ')

# import scipy as sp

# I = sp.sparse.csr_matrix(0.005*np.ones((n,n)))

# g.adjacency_matrix = g.adjacency_matrix + I

## Example on how to find all clusters in graph using local graph clustering

In [126]:
from sklearn.metrics import pairwise_distances

start = time.time()
# Compute the embeddings X and the pairwise distance matrix Z.
embeddings = compute_all_embeddings(g,rho_list=[1.0e-5,1.0e-1],alpha_list=[1.0e-1,2.0e-1],nsamples_from_rho=1,nsamples_from_alpha=1,njobs=6,normalized_objective=False,normalize=False)
# Z = pairwise_distances(X, metric='l2', n_jobs=1)
end = time.time()
print(end - start)

48.19090414047241


In [128]:
embeddings_local = normalize_embeddings(g,embeddings, norm_type = 2)
# start = time.time()
# # Find the clusters
# labels = compute_clusters_given_distance(nclusters=39,Z=Z)
# end = time.time()
# print(end - start)

In [135]:
from scipy.sparse.linalg import svds
u, s, vt = svds(embeddings_local, 200, which = 'LM')

In [136]:
groups = np.loadtxt('./datasets/BlogCatalog-dataset/data/group-edges.txt', dtype = 'int',delimiter=',') - 1

Y = np.zeros((g._num_vertices,39))

for data in groups:
    idx1 = data[0]
    idx2 = data[1]
    Y[idx1,idx2] = 1

In [137]:
# edgelist = np.loadtxt('./datasets/BlogCatalog-dataset/data/edges.txt', dtype = 'int',delimiter=',') - 1
# np.savetxt("./datasets/BlogCatalog-dataset/data/edges2.txt", edgelist, newline="\n", fmt='%d')

In [141]:
X_train = u[0:int(g._num_vertices/8)]
X_test  = u[int(g._num_vertices/8):]
Y_train = Y[0:int(g._num_vertices/8)]
Y_test  = Y[int(g._num_vertices/8):]

In [143]:
    print(__doc__)

    import numpy as np
    import matplotlib.pyplot as plt

    from sklearn.datasets import make_multilabel_classification
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    from sklearn.cross_decomposition import CCA
    from sklearn.metrics import accuracy_score

#     classif = OneVsRestClassifier(SVC(kernel='linear'))
#     classif.fit(X, Y)
    
    labels = OneVsRestClassifier(SVC(kernel='linear',C=10000,gamma='auto',shrinking=True,tol=1.0e-3,decision_function_shape='ovr',probability=False)).fit(X_train, Y_train).predict(X_test) 
    
    

Automatically created module for IPython interactive environment


/usr/local/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning:

Label not 37 is present in all training examples.



In [144]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

f1_score(labels, Y_test, average="macro")

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true samples.



0.12416113295852664

In [28]:
# n = g._num_vertices
# from sklearn.cluster import spectral_clustering as spclustering
# start = time.time()
# labels_spclustering = list(spclustering(g.adjacency_matrix.toarray() + 0.0005*np.ones((n,n)) , n_clusters=39, eigen_solver='arpack'))
# end = time.time()
# print(end - start)
# # for data in groups:
# #     idx1 = data[0]
# #     idx2 = data[1]
# #     group_member[idx1,idx2] = 1
# labels_spclustering = np.array(labels_spclustering, dtype=int)

# # groups_global = np.zeros((g._num_vertices,40))
# # for i in range(g._num_vertices):
# #     groups_global[i,labels_spclustering[i]] = 1

## Global spectral

In [57]:
from sklearn.manifold import spectral_embedding as spembedding
n = g._num_vertices
embeddings_global = spembedding(g.adjacency_matrix, n_components=2, eigen_solver='arpack', norm_laplacian=True, drop_first=True)

In [58]:
embeddings_global.shape

(10312, 2)

In [59]:
groups = np.loadtxt('./datasets/BlogCatalog-dataset/data/group-edges.txt', dtype = 'int',delimiter=',') - 1

Y = np.zeros((g._num_vertices,39))

for data in groups:
    idx1 = data[0]
    idx2 = data[1]
    Y[idx1,idx2] = 1

In [60]:
X_train = embeddings_global[0:int(g._num_vertices/8)]
X_test  = embeddings_global[int(g._num_vertices/8):]
Y_train = Y[0:int(g._num_vertices/8)]
Y_test  = Y[int(g._num_vertices/8):]

In [61]:
    print(__doc__)

    import numpy as np
    import matplotlib.pyplot as plt

    from sklearn.datasets import make_multilabel_classification
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    from sklearn.cross_decomposition import CCA
    from sklearn.metrics import accuracy_score

#     classif = OneVsRestClassifier(SVC(kernel='linear'))
#     classif.fit(X, Y)
    
    labels = OneVsRestClassifier(SVC(kernel='linear',C=10,gamma='auto',shrinking=False,tol=1.0e-3)).fit(X_train, Y_train).predict(X_test) 
    
    

Automatically created module for IPython interactive environment


/usr/local/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning:

Label not 37 is present in all training examples.



In [62]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

f1_score(labels, Y_test, average="macro")

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true samples.



0.0

## Node2vec

In [27]:
node2vec = np.loadtxt('/Users/kimonfountoulakis/Downloads/snap-master/examples/node2vec/emb/blog.emb', dtype = 'float',delimiter=' ')

In [28]:
groups = np.loadtxt('./datasets/BlogCatalog-dataset/data/group-edges.txt', dtype = 'int',delimiter=',') - 1

Y = np.zeros((g._num_vertices,39))

for data in groups:
    idx1 = data[0]
    idx2 = data[1]
    Y[idx1,idx2] = 1
        
sorted_idx = np.argsort(node2vec[:,0])
node2vec_emb = node2vec[sorted_idx,:][:, 1: ]
    
X_train = node2vec_emb[0:int(g._num_vertices/8)]
X_test  = node2vec_emb[int(g._num_vertices/8):]
Y_train = Y[0:int(g._num_vertices/8)]
Y_test  = Y[int(g._num_vertices/8):]

In [29]:
    print(__doc__)

    import numpy as np
    import matplotlib.pyplot as plt

    from sklearn.datasets import make_multilabel_classification
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    from sklearn.cross_decomposition import CCA
    from sklearn.metrics import accuracy_score

#     classif = OneVsRestClassifier(SVC(kernel='linear'))
#     classif.fit(X, Y)
        
    labels = OneVsRestClassifier(SVC(kernel='linear',C=10,gamma='auto',shrinking=False,tol=1.0e-3)).fit(X_train, Y_train).predict(X_test) 
    
    

Automatically created module for IPython interactive environment


/usr/local/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning:

Label not 37 is present in all training examples.



In [30]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

f1_score(labels, Y_test, average="macro")

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true samples.



0.15814244984588974